In [1]:
storage_account_name = "<Storage account name>"
storage_account_key = "<Storage account key>"
container = "<Container name>"

In [2]:
spark.conf.set("fs.azure.account.key.{0}.blob.core.windows.net".format(storage_account_name), storage_account_key)

In [3]:
dbutils.fs.mount(
 source = "wasbs://{0}@{1}.blob.core.windows.net".format(container, storage_account_name),
 mount_point = "/mnt/data",
 extra_configs = {"fs.azure.account.key.{0}.blob.core.windows.net".format(storage_account_name): storage_account_key}
)

--------------------------------------------------------------------------- 
 ExecutionError Traceback (most recent call last)
 <command-3137789824692736> in <module> () 
 2 source = "wasbs://{0}@{1}.blob.core.windows.net" . format ( container , storage_account_name ) , 
 3 mount_point = "/mnt/data" , 
 ----> 4 extra_configs = { "fs.azure.account.key.{0}.blob.core.windows.net" . format ( storage_account_name ) : storage_account_key } 
 5 )

 /local_disk0/tmp/1561932344064-0/dbutils.py in f_with_exception_handling (*args, **kwargs) 
 302 exc . __context__ = None 
 303 exc . __cause__ = None 
 --> 304 raise exc 
 305 return f_with_exception_handling 
 306 

 ExecutionError : An error occurred while calling o263.mount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/data; nested exception is: 
	java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/data
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:100)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:54)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.mount(DBUtilsCore.scala:454)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/data
	at scala.Predef$.require(Predef.scala:281)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.$anonfun$insertMount$1(MetadataManager.scala:160)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.withRetries(MetadataManager.scala:206)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.insertMount(MetadataManager.scala:154)
	at com.databricks.backend.daemon.data.server.handler.MountHandler.receive(MountHandler.scala:79)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1(SessionContext.scala:103)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1$adapted(SessionContext.scala:102)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at com.databricks.backend.daemon.data.server.session.SessionContext.queryHandlers(SessionContext.scala:102)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$3.applyOrElse(DbfsServerBackend.scala:298)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$3.applyOrElse(DbfsServerBackend.scala:276)
	at com.databricks.rpc.ServerBackend.$anonfun$internalReceive$2(ServerBackend.scala:44)
	at com.databricks.rpc.ServerBackend$$anonfun$commonReceive$1.applyOrElse(ServerBackend.scala:61)
	at com.databricks.rpc.ServerBackend$$anonfun$commonReceive$1.applyOrElse(ServerBackend.scala:61)
	at com.databricks.rpc.ServerBackend.$anonfun$internalReceive$1(ServerBackend.scala:40)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$2(UsageLogging.scala:369)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:238)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:233)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:230)
	at com.databricks.rpc.Ser

In [4]:
%fs ls /mnt/data

path,name,size
dbfs:/mnt/data/housing.csv,housing.csv,1423529


In [5]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator

In [6]:
data = spark.read \
  .option("header", "true") \
  .option("inferSchema", "true") \
  .option("delimiter", ",") \
  .csv("/mnt/data/housing.csv".format(container, storage_account_name))

data.show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
 -122.23| 37.88| 41.0| 880.0| 129.0| 322.0| 126.0| 8.3252| 452600.0| NEAR BAY|
 -122.22| 37.86| 21.0| 7099.0| 1106.0| 2401.0| 1138.0| 8.3014| 358500.0| NEAR BAY|
 -122.24| 37.85| 52.0| 1467.0| 190.0| 496.0| 177.0| 7.2574| 352100.0| NEAR BAY|
 -122.25| 37.85| 52.0| 1274.0| 235.0| 558.0| 219.0| 5.6431| 341300.0| NEAR BAY|
 -122.25| 37.85| 52.0| 1627.0| 280.0| 565.0| 259.0| 3.8462| 342200.0| NEAR BAY|
 -122.25| 37.85| 52.0| 919.0| 213.0| 413.0| 193.0| 4.0368| 269700.0| NEAR BAY|
 -122.25| 37.84| 52.0| 2535.0| 489.0| 1094.0| 514.0| 3.6591| 299200.0| NEAR BAY|
 -122.25| 37.84| 52.0| 3104.0| 687.0| 1157.0| 647.0| 3.12| 241400.0| NEAR BAY|
 -122.26| 37.84| 42.0| 2555.0| 665.0| 1206.0| 595.0| 2.0804| 226700.0| NEAR BAY|
 -122.25| 37.84| 52.0| 3549.0| 707.0| 1551.0| 714.0| 3.6912| 261100.0| NEAR BAY|
 -122.26| 37.85| 52.0| 2202.0| 434.0| 910.0| 402.0| 3.2031| 281500.0| NEAR BAY|
 -122.26| 37.85| 52.0| 3503.0| 752.0| 1504.0| 734.0| 3.2705| 241800.0| NEAR BAY|
 -122.26| 37.85| 52.0| 2491.0| 474.0| 1098.0| 468.0| 3.075| 213500.0| NEAR BAY|
 -122.26| 37.84| 52.0| 696.0| 191.0| 345.0| 174.0| 2.6736| 191300.0| NEAR BAY|
 -122.26| 37.85| 52.0| 2643.0| 626.0| 1212.0| 620.0| 1.9167| 159200.0| NEAR BAY|
 -122.26| 37.85| 50.0| 1120.0| 283.0| 697.0| 264.0| 2.125| 140000.0| NEAR BAY|
 -122.27| 37.85| 52.0| 1966.0| 347.0| 793.0| 331.0| 2.775| 152500.0| NEAR BAY|
 -122.27| 37.85| 52.0| 1228.0| 293.0| 648.0| 303.0| 2.1202| 155500.0| NEAR BAY|
 -122.26| 37.84| 50.0| 2239.0| 455.0| 990.0| 419.0| 1.9911| 158700.0| NEAR BAY|
 -122.27| 37.84| 52.0| 1503.0| 298.0| 690.0| 275.0| 2.6033| 162900.0| NEAR BAY|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
only showing top 20 rows

In [7]:
vectors = VectorAssembler(inputCols= ['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income'], outputCol= 'features')

vectors.setParams(handleInvalid="skip")

vector_data = vectors.transform(data)

Out[ 7 ]: VectorAssembler_e34e9920db4c

In [8]:
features = vector_data.select(["features", "median_house_value"])

features.show()

+--------------------+------------------+
 features|median_house_value|
+--------------------+------------------+
[-122.23,37.88,41...| 452600.0|
[-122.22,37.86,21...| 358500.0|
[-122.24,37.85,52...| 352100.0|
[-122.25,37.85,52...| 341300.0|
[-122.25,37.85,52...| 342200.0|
[-122.25,37.85,52...| 269700.0|
[-122.25,37.84,52...| 299200.0|
[-122.25,37.84,52...| 241400.0|
[-122.26,37.84,42...| 226700.0|
[-122.25,37.84,52...| 261100.0|
[-122.26,37.85,52...| 281500.0|
[-122.26,37.85,52...| 241800.0|
[-122.26,37.85,52...| 213500.0|
[-122.26,37.84,52...| 191300.0|
[-122.26,37.85,52...| 159200.0|
[-122.26,37.85,50...| 140000.0|
[-122.27,37.85,52...| 152500.0|
[-122.27,37.85,52...| 155500.0|
[-122.26,37.84,50...| 158700.0|
[-122.27,37.84,52...| 162900.0|
+--------------------+------------------+
only showing top 20 rows

In [9]:
(trainingData, testData) = features.randomSplit([0.7, 0.3])

In [10]:
lr = LinearRegression(labelCol="median_house_value", featuresCol="features")

In [11]:
model = lr.fit(trainingData)

In [12]:
print("Coefficients:", model.coefficients)
print("Intercept:", model.intercept)

Coefficients: [-42730.1204457,-42509.7369354,1157.90030725,-8.24972507065,113.820707125,-38.3855780499,47.701351343,40297.5217158]
Intercept: -3585395.747191204

In [13]:
model_summary = model.summary

print("RMSE", model_summary.rootMeanSquaredError)
print("R^2:", model_summary.r2)

RMSE 69966.70329855316
R^2: 0.6330400246135165

In [14]:
evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="median_house_value", metricName="r2")

In [15]:
test_data = model.transform(testData)

evaluator.evaluate(test_data)

Out[ 24 ]: 0.6458585401152135